In [ ]:
# Import dependencies
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
# Loading data

In [ ]:
# # Create a OneHotEncoder instance
# enc = OneHotEncoder(sparse=False)

# # Fit and transform the OneHotEncoder using the categorical variable list
# encode_df = pd.DataFrame(enc.fit_transform(df[df]))

# # Add the encoded variable names to the DataFrame
# encode_df.columns = enc.get_feature_names(df)
# encode_df.head()

In [ ]:
# # Merge one-hot encoded features and drop the originals
# df = df.merge(encode_df,left_index=True, right_index=True)
# df = df.drop(df,1)
# df.head()

In [ ]:
# # Define the features set.
# X = 

# # Define the target set.
# y = 

In [ ]:
# # Splitting into Train and Test sets.
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# # Creating a StandardScaler instance.
# scaler = StandardScaler()
# # Fitting the Standard Scaler with the training data.
# X_scaler = scaler.fit(X_train)

# # Scaling the data.
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# # Create a random forest classifier.
# rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

In [ ]:
# # Fitting the model
# rf_model = rf_model.fit(X_train_scaled, y_train)

In [ ]:
# # Making predictions using the testing data.
# predictions = rf_model.predict(X_test_scaled)

In [ ]:
# # Calculating the confusion matrix.
# cm = confusion_matrix(y_test, predictions)

# # Create a DataFrame from the confusion matrix.
# cm_df = pd.DataFrame(
#     cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

# cm_df

In [ ]:
# # Calculating the accuracy score.
# acc_score = accuracy_score(y_test, predictions)

In [ ]:
# # Displaying results
# print("Confusion Matrix")
# display(cm_df)
# print(f"Accuracy Score : {acc_score}")
# print("Classification Report")
# print(classification_report(y_test, predictions))

In [ ]:
# # Calculate feature importance in the Random Forest model.
# importances = rf_model.feature_importances_
# importances

In [ ]:
# # We can sort the features by their importance.
# sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)